# caQTL 2

We previously identified a proper subset of our data, let's now follow the rest of chrombpnet preprocessing

In [ ]:
#chrombpnet subsets to only peaks, we can have one that is generated as peaks vs not I guess?